##### Copyright 2018 Les auteurs de TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/saved_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Voir la source sur GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/saved_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Télécharger le cahier</a></td>
</table>

Un SavedModel contient un programme TensorFlow complet, y compris les pondérations et le calcul. Il ne nécessite pas l'exécution du code de construction du modèle d'origine, ce qui le rend utile pour le partage ou le déploiement (avec [TFLite](https://tensorflow.org/lite) , [TensorFlow.js](https://js.tensorflow.org/) , [TensorFlow Serving](https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple) ou [TensorFlow Hub](https://tensorflow.org/hub) ).

Ce document plonge dans certains des détails de l'utilisation de l'API de bas niveau `tf.saved_model` :

- Si vous utilisez un `tf.keras.Model` la `keras.Model.save(output_path)` peut être tout ce dont vous avez besoin : Voir la [sauvegarde et la sérialisation de Keras](keras/save_and_serialize.ipynb)

- Si vous souhaitez simplement enregistrer/charger des poids pendant l'entraînement, consultez le [guide des points de contrôle de l'entraînement](./checkpoint.ipynb) .


## Création d'un modèle enregistré à partir de Keras

Pour une introduction rapide, cette section exporte un modèle Keras pré-formé et sert les demandes de classification d'images avec celui-ci. Le reste du guide fournira des détails et discutera d'autres façons de créer des SavedModels.

In [0]:
import os
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

tmpdir = tempfile.mkdtemp()

In [0]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [0]:
file = tf.keras.utils.get_file(
    "grace_hopper.jpg",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg")
img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
plt.imshow(img)
plt.axis('off')
x = tf.keras.preprocessing.image.img_to_array(img)
x = tf.keras.applications.mobilenet.preprocess_input(
    x[tf.newaxis,...])

<a id="saved_model_cli"></a>

## Détails de l'interface de ligne de commande SavedModel

Vous pouvez utiliser l'interface de ligne de commande (CLI) SavedModel pour inspecter et exécuter un SavedModel. Par exemple, vous pouvez utiliser la CLI pour inspecter les `SignatureDef` s du modèle. La CLI vous permet de confirmer rapidement que le dtype et la forme du tenseur d'entrée correspondent au modèle. De plus, si vous souhaitez tester votre modèle, vous pouvez utiliser la CLI pour effectuer une vérification de cohérence en transmettant des exemples d'entrées dans différents formats (par exemple, des expressions Python), puis en récupérant la sortie.

### Installer l'interface de ligne de commande SavedModel

D'une manière générale, vous pouvez installer TensorFlow de l'une des deux manières suivantes :

- En installant un binaire TensorFlow pré-construit.
- En créant TensorFlow à partir du code source.

Si vous avez installé TensorFlow via un binaire TensorFlow pré-construit, la CLI SavedModel est déjà installée sur votre système au nom de chemin `bin/saved_model_cli` .

Si vous avez compilé TensorFlow à partir du code source, vous devez exécuter la commande supplémentaire suivante pour compiler `saved_model_cli` :

```
$ bazel build tensorflow/python/tools:saved_model_cli
```

### Présentation des commandes

La CLI SavedModel prend en charge les deux commandes suivantes sur un SavedModel :

- `show` , qui affiche les calculs disponibles à partir d'un SavedModel.
- `run` , qui exécute un calcul à partir d'un SavedModel.

### `show` la commande

Un SavedModel contient une ou plusieurs variantes de modèle (techniquement, `v1.MetaGraphDef` s), identifiées par leurs ensembles de balises. Pour servir un modèle, vous pourriez vous demander quel type de `SignatureDef` s sont dans chaque variante de modèle, et quelles sont leurs entrées et sorties. La commande `show` vous permet d'examiner le contenu du SavedModel dans l'ordre hiérarchique. Voici la syntaxe :

```
usage: saved_model_cli show [-h] --dir DIR [--all]
[--tag_set TAG_SET] [--signature_def SIGNATURE_DEF_KEY]
```

Par exemple, la commande suivante affiche tous les ensembles de balises disponibles dans le SavedModel :

```
$ saved_model_cli show --dir /tmp/saved_model_dir
The given SavedModel contains the following tag-sets:
serve
serve, gpu
```

La commande suivante affiche toutes les clés `SignatureDef` disponibles pour un ensemble de balises :

```
$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve
The given SavedModel `MetaGraphDef` contains `SignatureDefs` with the
following keys:
SignatureDef key: "classify_x2_to_y3"
SignatureDef key: "classify_x_to_y"
SignatureDef key: "regress_x2_to_y3"
SignatureDef key: "regress_x_to_y"
SignatureDef key: "regress_x_to_y2"
SignatureDef key: "serving_default"
```

S'il y a *plusieurs* balises dans le jeu de balises, vous devez spécifier toutes les balises, chaque balise étant séparée par une virgule. Par example:

<pre>$ saved_model_cli show --dir /tmp/saved_model_dir --tag_set serve,gpu
</pre>

Pour afficher toutes les entrées et sorties TensorInfo pour un `SignatureDef` spécifique, passez la clé `SignatureDef` à l'option `signature_def` . Ceci est très utile lorsque vous souhaitez connaître la valeur de la clé du tenseur, le dtype et la forme des tenseurs d'entrée pour exécuter le graphe de calcul ultérieurement. Par example:

```
$ saved_model_cli show --dir \
/tmp/saved_model_dir --tag_set serve --signature_def serving_default
The given SavedModel SignatureDef contains the following input(s):
  inputs['x'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: x:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['y'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: y:0
Method name is: tensorflow/serving/predict
```

Pour afficher toutes les informations disponibles dans SavedModel, utilisez l'option `--all` . Par example:

<pre>$ saved_model_cli show --dir /tmp/saved_model_dir --all
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classify_x2_to_y3']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y3:0
  Method name is: tensorflow/serving/classify

...

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['x'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['y'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y:0
  Method name is: tensorflow/serving/predict
</pre>

### `run` commande

Appelez la commande `run` pour exécuter un calcul de graphe, en transmettant les entrées, puis en affichant (et éventuellement en enregistrant) les sorties. Voici la syntaxe :

```
usage: saved_model_cli run [-h] --dir DIR --tag_set TAG_SET --signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]
```

La commande `run` fournit les trois manières suivantes de transmettre des entrées au modèle :

- L'option `--inputs` vous permet de passer numpy ndarray dans les fichiers.
- L'option `--input_exprs` vous permet de transmettre des expressions Python.
- L'option `--input_examples` vous permet de passer `tf.train.Example` .

#### `--inputs`

Pour transmettre des données d'entrée dans des fichiers, spécifiez l'option `--inputs` , qui prend le format général suivant :

```bsh
--inputs <INPUTS>
```

où *INPUTS* est l'un des formats suivants :

- `<input_key>=<filename>`
- `<input_key>=<filename>[<variable_name>]`

Vous pouvez passer plusieurs *INPUTS* . Si vous transmettez plusieurs entrées, utilisez un point-virgule pour séparer chacune des *INPUTS* .

`saved_model_cli` utilise `numpy.load` pour charger le *filename* . Le nom de *fichier* peut être dans l'un des formats suivants :

- `.npy`
- `.npz`
- format cornichon

Un fichier `.npy` contient toujours un ndarray numpy. Par conséquent, lors du chargement à partir d'un fichier `.npy` , le contenu sera directement affecté au tenseur d'entrée spécifié. Si vous spécifiez un *variable_name* avec ce fichier `.npy` , le *variable_name* sera ignoré et un avertissement sera émis.

Lors du chargement à partir d'un `.npz` (zip), vous pouvez éventuellement spécifier un *variable_name* pour identifier la variable dans le fichier zip à charger pour la clé de tenseur d'entrée. Si vous ne spécifiez pas de *variable_name* , la CLI SavedModel vérifiera qu'un seul fichier est inclus dans le fichier zip et le chargera pour la clé de tenseur d'entrée spécifiée.

Lors du chargement à partir d'un fichier pickle, si aucun `variable_name` n'est spécifié entre crochets, tout ce qui se trouve à l'intérieur du fichier pickle sera transmis à la clé de tenseur d'entrée spécifiée. Sinon, la CLI SavedModel supposera qu'un dictionnaire est stocké dans le fichier pickle et la valeur correspondant au *variable_name* sera utilisée.

#### `--input_exprs`

Pour transmettre des entrées via des expressions Python, spécifiez l'option `--input_exprs` . Cela peut être utile lorsque vous n'avez pas de fichiers de données, mais que vous souhaitez tout de même vérifier l'intégrité du modèle avec quelques entrées simples qui correspondent au dtype et à la forme des `SignatureDef` s du modèle. Par example:

```bsh
`<input_key>=[[1],[2],[3]]`
```

En plus des expressions Python, vous pouvez également passer des fonctions numpy. Par example:

```bsh
`<input_key>=np.ones((32,32,3))`
```

(Notez que le module `numpy` est déjà disponible pour vous en tant que `np` .)

#### `--input_examples`

Pour passer `tf.train.Example` comme entrées, spécifiez l'option `--input_examples` . Pour chaque clé d'entrée, il faut une liste de dictionnaires, où chaque dictionnaire est une instance de `tf.train.Example` . Les clés du dictionnaire sont les fonctionnalités et les valeurs sont les listes de valeurs pour chaque fonctionnalité. Par example:

```bsh
`<input_key>=[{"age":[22,24],"education":["BS","MS"]}]`
```

#### Enregistrer la sortie

Par défaut, la CLI SavedModel écrit la sortie sur stdout. Si un répertoire est passé à l'option `--outdir` , les sorties seront enregistrées en tant que fichiers `.npy` nommés d'après les clés de tenseur de sortie sous le répertoire donné.

Utilisez `--overwrite` pour écraser les fichiers de sortie existants.
